In [ ]:
import warnings
warnings.simplefilter("ignore", UserWarning)
import requests
import spacy
nlp = spacy.load('en_core_web_sm') 
import nltk
from nltk.corpus import stopwords
from nltk import word_tokenize, pos_tag, ne_chunk
from nltk import RegexpParser
from nltk import Tree
nltk.download('stopwords')
stop_words_l=stopwords.words('english')
import pandas as pd
import numpy as np
import re
from sklearn.metrics.pairwise import cosine_similarity
from __future__ import unicode_literals
import en_core_web_sm
from spacy.matcher import Matcher
from spacy.util import filter_spans
from nltk.tokenize import word_tokenize

Creating run_mlm.py file to train data

Credits: https://github.com/huggingface/transformers/blob/master/examples/language-modeling/run_mlm.py

In [2]:
s='''# coding=utf-8
# Copyright 2020 The HuggingFace Team All rights reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
"""
Fine-tuning the library models for masked language modeling (BERT, ALBERT, RoBERTa...) on a text file or a dataset.
Here is the full list of checkpoints on the hub that can be fine-tuned by this script:
https://huggingface.co/models?filter=masked-lm
"""
# You can also adapt this script on your own masked language modeling task. Pointers for this are left as comments.

import logging
import math
import os
import sys
from dataclasses import dataclass, field
from typing import Optional
from sklearn.metrics import accuracy_score

from datasets import load_dataset

import transformers
from transformers import (
    CONFIG_MAPPING,
    MODEL_FOR_MASKED_LM_MAPPING,
    AutoConfig,
    AutoModelForMaskedLM,
    AutoTokenizer,
    DataCollatorForLanguageModeling,
    HfArgumentParser,
    Trainer,
    TrainingArguments,
    set_seed,
)
from transformers.trainer_utils import is_main_process


logger = logging.getLogger(__name__)
MODEL_CONFIG_CLASSES = list(MODEL_FOR_MASKED_LM_MAPPING.keys())
MODEL_TYPES = tuple(conf.model_type for conf in MODEL_CONFIG_CLASSES)

def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    acc = accuracy_score(labels, preds)
    return {
        'accuracy': acc
    }


@dataclass
class ModelArguments:
    """
    Arguments pertaining to which model/config/tokenizer we are going to fine-tune, or train from scratch.
    """

    model_name_or_path: Optional[str] = field(
        default=None,
        metadata={
            "help": "The model checkpoint for weights initialization."
            "Don't set if you want to train a model from scratch."
        },
    )
    model_type: Optional[str] = field(
        default=None,
        metadata={"help": "If training from scratch, pass a model type from the list: " + ", ".join(MODEL_TYPES)},
    )
    config_name: Optional[str] = field(
        default=None, metadata={"help": "Pretrained config name or path if not the same as model_name"}
    )
    tokenizer_name: Optional[str] = field(
        default=None, metadata={"help": "Pretrained tokenizer name or path if not the same as model_name"}
    )
    cache_dir: Optional[str] = field(
        default=None,
        metadata={"help": "Where do you want to store the pretrained models downloaded from huggingface.co"},
    )
    use_fast_tokenizer: bool = field(
        default=True,
        metadata={"help": "Whether to use one of the fast tokenizer (backed by the tokenizers library) or not."},
    )


@dataclass
class DataTrainingArguments:
    """
    Arguments pertaining to what data we are going to input our model for training and eval.
    """

    dataset_name: Optional[str] = field(
        default=None, metadata={"help": "The name of the dataset to use (via the datasets library)."}
    )
    dataset_config_name: Optional[str] = field(
        default=None, metadata={"help": "The configuration name of the dataset to use (via the datasets library)."}
    )
    train_file: Optional[str] = field(default=None, metadata={"help": "The input training data file (a text file)."})
    validation_file: Optional[str] = field(
        default=None,
        metadata={"help": "An optional input evaluation data file to evaluate the perplexity on (a text file)."},
    )
    overwrite_cache: bool = field(
        default=False, metadata={"help": "Overwrite the cached training and evaluation sets"}
    )
    max_seq_length: Optional[int] = field(
        default=None,
        metadata={
            "help": "The maximum total input sequence length after tokenization. Sequences longer "
            "than this will be truncated."
        },
    )
    preprocessing_num_workers: Optional[int] = field(
        default=None,
        metadata={"help": "The number of processes to use for the preprocessing."},
    )
    mlm_probability: float = field(
        default=0.15, metadata={"help": "Ratio of tokens to mask for masked language modeling loss"}
    )
    line_by_line: bool = field(
        default=False,
        metadata={"help": "Whether distinct lines of text in the dataset are to be handled as distinct sequences."},
    )
    pad_to_max_length: bool = field(
        default=False,
        metadata={
            "help": "Whether to pad all samples to `max_seq_length`. "
            "If False, will pad the samples dynamically when batching to the maximum length in the batch."
        },
    )

    def __post_init__(self):
        if self.dataset_name is None and self.train_file is None and self.validation_file is None:
            raise ValueError("Need either a dataset name or a training/validation file.")
        else:
            if self.train_file is not None:
                extension = self.train_file.split(".")[-1]
                assert extension in ["csv", "json", "txt"], "`train_file` should be a csv, a json or a txt file."
            if self.validation_file is not None:
                extension = self.validation_file.split(".")[-1]
                assert extension in ["csv", "json", "txt"], "`validation_file` should be a csv, a json or a txt file."


def main():
    # See all possible arguments in src/transformers/training_args.py
    # or by passing the --help flag to this script.
    # We now keep distinct sets of args, for a cleaner separation of concerns.

    parser = HfArgumentParser((ModelArguments, DataTrainingArguments, TrainingArguments))
    if len(sys.argv) == 2 and sys.argv[1].endswith(".json"):
        # If we pass only one argument to the script and it's the path to a json file,
        # let's parse it to get our arguments.
        model_args, data_args, training_args = parser.parse_json_file(json_file=os.path.abspath(sys.argv[1]))
    else:
        model_args, data_args, training_args = parser.parse_args_into_dataclasses()

    if (
        os.path.exists(training_args.output_dir)
        and os.listdir(training_args.output_dir)
        and training_args.do_train
        and not training_args.overwrite_output_dir
    ):
        raise ValueError(
            f"Output directory ({training_args.output_dir}) already exists and is not empty."
            "Use --overwrite_output_dir to overcome."
        )

    # Setup logging
    logging.basicConfig(
        format="%(asctime)s - %(levelname)s - %(name)s -   %(message)s",
        datefmt="%m/%d/%Y %H:%M:%S",
        level=logging.INFO if is_main_process(training_args.local_rank) else logging.WARN,
    )

    # Log on each process the small summary:
    logger.warning(
        f"Process rank: {training_args.local_rank}, device: {training_args.device}, n_gpu: {training_args.n_gpu}"
        + f"distributed training: {bool(training_args.local_rank != -1)}, 16-bits training: {training_args.fp16}"
    )
    # Set the verbosity to info of the Transformers logger (on main process only):
    if is_main_process(training_args.local_rank):
        transformers.utils.logging.set_verbosity_info()
        transformers.utils.logging.enable_default_handler()
        transformers.utils.logging.enable_explicit_format()
    logger.info("Training/evaluation parameters %s", training_args)

    # Set seed before initializing model.
    set_seed(training_args.seed)

    # Get the datasets: you can either provide your own CSV/JSON/TXT training and evaluation files (see below)
    # or just provide the name of one of the public datasets available on the hub at https://huggingface.co/datasets/
    # (the dataset will be downloaded automatically from the datasets Hub
    #
    # For CSV/JSON files, this script will use the column called 'text' or the first column. You can easily tweak this
    # behavior (see below)
    #
    # In distributed training, the load_dataset function guarantee that only one local process can concurrently
    # download the dataset.
    if data_args.dataset_name is not None:
        # Downloading and loading a dataset from the hub.
        datasets = load_dataset(data_args.dataset_name, data_args.dataset_config_name)
    else:
        data_files = {}
        if data_args.train_file is not None:
            data_files["train"] = data_args.train_file
        if data_args.validation_file is not None:
            data_files["validation"] = data_args.validation_file
        extension = data_args.train_file.split(".")[-1]
        if extension == "txt":
            extension = "text"
        datasets = load_dataset(extension, data_files=data_files)
    # See more about loading any type of standard or custom dataset (from files, python dict, pandas DataFrame, etc) at
    # https://huggingface.co/docs/datasets/loading_datasets.html.

    # Load pretrained model and tokenizer
    #
    # Distributed training:
    # The .from_pretrained methods guarantee that only one local process can concurrently
    # download model & vocab.
    if model_args.config_name:
        config = AutoConfig.from_pretrained(model_args.config_name, cache_dir=model_args.cache_dir)
    elif model_args.model_name_or_path:
        config = AutoConfig.from_pretrained(model_args.model_name_or_path, cache_dir=model_args.cache_dir)
    else:
        config = CONFIG_MAPPING[model_args.model_type]()
        logger.warning("You are instantiating a new config instance from scratch.")

    if model_args.tokenizer_name:
        tokenizer = AutoTokenizer.from_pretrained(
            model_args.tokenizer_name, cache_dir=model_args.cache_dir, use_fast=model_args.use_fast_tokenizer
        )
    elif model_args.model_name_or_path:
        tokenizer = AutoTokenizer.from_pretrained(
            model_args.model_name_or_path, cache_dir=model_args.cache_dir, use_fast=model_args.use_fast_tokenizer
        )
    else:
        raise ValueError(
            "You are instantiating a new tokenizer from scratch. This is not supported by this script."
            "You can do it from another script, save it, and load it from here, using --tokenizer_name."
        )

    if model_args.model_name_or_path:
        model = AutoModelForMaskedLM.from_pretrained(
            model_args.model_name_or_path,
            from_tf=bool(".ckpt" in model_args.model_name_or_path),
            config=config,
            cache_dir=model_args.cache_dir,
        )
    else:
        logger.info("Training new model from scratch")
        model = AutoModelForMaskedLM.from_config(config)

    model.resize_token_embeddings(len(tokenizer))

    # Preprocessing the datasets.
    # First we tokenize all the texts.
    if training_args.do_train:
        column_names = datasets["train"].column_names
    else:
        column_names = datasets["validation"].column_names
    text_column_name = "text" if "text" in column_names else column_names[0]

    if data_args.line_by_line:
        # When using line_by_line, we just tokenize each nonempty line.
        padding = "max_length" if data_args.pad_to_max_length else False

        def tokenize_function(examples):
            # Remove empty lines
            examples["text"] = [line for line in examples["text"] if len(line) > 0 and not line.isspace()]
            return tokenizer(
                examples["text"],
                padding=padding,
                truncation=True,
                max_length=data_args.max_seq_length,
                # We use this option because DataCollatorForLanguageModeling (see below) is more efficient when it
                # receives the `special_tokens_mask`.
                return_special_tokens_mask=True,
            )

        tokenized_datasets = datasets.map(
            tokenize_function,
            batched=True,
            num_proc=data_args.preprocessing_num_workers,
            remove_columns=[text_column_name],
            load_from_cache_file=not data_args.overwrite_cache,
        )
    else:
        # Otherwise, we tokenize every text, then concatenate them together before splitting them in smaller parts.
        # We use `return_special_tokens_mask=True` because DataCollatorForLanguageModeling (see below) is more
        # efficient when it receives the `special_tokens_mask`.
        def tokenize_function(examples):
            return tokenizer(examples[text_column_name], return_special_tokens_mask=True)

        tokenized_datasets = datasets.map(
            tokenize_function,
            batched=True,
            num_proc=data_args.preprocessing_num_workers,
            remove_columns=column_names,
            load_from_cache_file=not data_args.overwrite_cache,
        )

        if data_args.max_seq_length is None:
            max_seq_length = tokenizer.model_max_length
        else:
            if data_args.max_seq_length > tokenizer.model_max_length:
                logger.warn(
                    f"The max_seq_length passed ({data_args.max_seq_length}) is larger than the maximum length for the"
                    f"model ({tokenizer.model_max_length}). Using max_seq_length={tokenizer.model_max_length}."
                )
            max_seq_length = min(data_args.max_seq_length, tokenizer.model_max_length)

        # Main data processing function that will concatenate all texts from our dataset and generate chunks of
        # max_seq_length.
        def group_texts(examples):
            # Concatenate all texts.
            concatenated_examples = {k: sum(examples[k], []) for k in examples.keys()}
            total_length = len(concatenated_examples[list(examples.keys())[0]])
            # We drop the small remainder, we could add padding if the model supported it instead of this drop, you can
            # customize this part to your needs.
            total_length = (total_length // max_seq_length) * max_seq_length
            # Split by chunks of max_len.
            result = {
                k: [t[i : i + max_seq_length] for i in range(0, total_length, max_seq_length)]
                for k, t in concatenated_examples.items()
            }
            return result

        # Note that with `batched=True`, this map processes 1,000 texts together, so group_texts throws away a
        # remainder for each of those groups of 1,000 texts. You can adjust that batch_size here but a higher value
        # might be slower to preprocess.
        #
        # To speed up this part, we use multiprocessing. See the documentation of the map method for more information:
        # https://huggingface.co/docs/datasets/package_reference/main_classes.html#datasets.Dataset.map
        tokenized_datasets = tokenized_datasets.map(
            group_texts,
            batched=True,
            num_proc=data_args.preprocessing_num_workers,
            load_from_cache_file=not data_args.overwrite_cache,
        )

    # Data collator
    # This one will take care of randomly masking the tokens.
    data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm_probability=data_args.mlm_probability)

    # Initialize our Trainer
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=tokenized_datasets["train"] if training_args.do_train else None,
        eval_dataset=tokenized_datasets["validation"] if training_args.do_eval else None,
        tokenizer=tokenizer,
        data_collator=data_collator,
    )

    # Training
    if training_args.do_train:
        model_path = (
            model_args.model_name_or_path
            if (model_args.model_name_or_path is not None and os.path.isdir(model_args.model_name_or_path))
            else None
        )
        trainer.train(model_path=model_path)
        trainer.save_model()  # Saves the tokenizer too for easy upload

    # Evaluation
    results = {}
    if training_args.do_eval:
        logger.info("*** Evaluate ***")

        eval_output = trainer.evaluate()

        perplexity = math.exp(eval_output["eval_loss"])
        results["perplexity"] = perplexity

        output_eval_file = os.path.join(training_args.output_dir, "eval_results_mlm.txt")
        if trainer.is_world_process_zero():
            with open(output_eval_file, "w") as writer:
                logger.info("***** Eval results *****")
                for key, value in results.items():
                    logger.info(f"  {key} = {value}")
                    writer.write(f"{key} = {value}")

    return results


def _mp_fn(index):
    # For xla_spawn (TPUs)
    main()


if __name__ == "__main__":
    main()'''


Creating training Python file.

In [3]:
f=open('run_mlm.py','w')
f.write(s)
f.close()

Installing dependencies.

In [ ]:
!pip install datasets

In [ ]:
!pip install transformers
!pip install fitbert

In [ ]:
!pip install dill==0.2.7.1

In [ ]:
!pip install sentence-transformers

In [ ]:
!pip install \
   pandas matplotlib numpy \
   nltk seaborn sklearn gensim pyldavis \
   wordcloud textblob spacy textstat

In [ ]:
import torch
import pandas as pd
import numpy as np
from nltk.corpus import stopwords
import nltk
nltk.download('stopwords')
import re
import csv
from sklearn.feature_extraction.text import TfidfVectorizer 
from sklearn.metrics.pairwise import cosine_similarity
stop_words_l=stopwords.words('english')
nlp = spacy.load('en_core_web_sm')
from nltk.corpus import stopwords
stop = stopwords.words('english')

In [ ]:
import dill
dill.dill._reverse_typemap['ClassType'] = type

In [ ]:
dill.__version__

Modifying some dependencies.

In [ ]:
f=open('/usr/local/lib/python3.6/dist-packages/datasets/utils/py_utils.py')
r=f.read()
r=r.replace('_dill','dill')
f.close()
f=open('/usr/local/lib/python3.6/dist-packages/datasets/utils/py_utils.py','w')
f.write(r)
f.close()

In [ ]:
from sentence_transformers import SentenceTransformer
sbert_model = SentenceTransformer('bert-base-nli-mean-tokens')

In [ ]:
!pip install sentence-transformers
from sentence_transformers import SentenceTransformer
sbert_model = SentenceTransformer('bert-base-nli-mean-tokens')

In [ ]:
!wget https://conceptnet.s3.amazonaws.com/downloads/2019/numberbatch/numberbatch-en-19.08.txt.gz
!python -m spacy init-model en /tmp/concept_net_embeddings --vectors-loc numberbatch-en-19.08.txt.gz

In [ ]:
# from transformers import BertTokenizer
# tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

In [ ]:
!cp -r drive/My\ Drive/recam/clean_data/* .

In [ ]:
concept_net = spacy.load("/tmp/concept_net_embeddings")

In [ ]:
sentence1 = concept_net("exploded")
sentence2 = concept_net("unconnected")
sentence1.similarity(sentence2)

In [ ]:
url = "http://api.conceptnet.io/c/en/@word?filter=/c/en"

In [ ]:
response = requests.get(url.replace("@word", "angry"))
obj = response.json()
string = obj['edges'][2]['surfaceText']
string = string.replace("[[", "")
string = string.replace("]]", "")
string

In [ ]:
def get_nth_key(dictionary, n=0):
    if n < 0:
        n += len(dictionary)
    for i, key in enumerate(dictionary.keys()):
        if i == n:
            return key
    raise IndexError("dictionary index out of range") 

def extract_n_v_phrase(sim_sent):
    for i, val in enumerate(sim_sent):
        nvp = []
        doc = nlp(val)
        for token in doc.noun_chunks: 
            nvp.append(token.text)

        pattern = [{'POS': 'VERB', 'OP': '?'},{'POS': 'ADV', 'OP': '*'},{'POS': 'AUX', 'OP': '*'},{'POS': 'VERB', 'OP': '+'}]
        matcher = Matcher(nlp.vocab)
        matcher.add("Verb phrase", None, pattern)
        matches = matcher(doc)
        spans = [doc[start:end] for _, start, end in matches]
        vps = filter_spans(spans)

        for vp in vps:
            nvp.append(vp.text)
            
        similarities = {}
        if(len(nvp)==0):
            return []
        for idx, x in enumerate(nvp):
            sen1 = concept_net(val)
            sen2 = concept_net(x)
            sim = sen1.similarity(sen2)
            similarities[sim] = idx
        sort_sim = sorted(similarities, reverse = True)
        result = []
        try:
            result.append(nvp[similarities[sort_sim[0]]])
            if(len(nvp)>1):
                result.append(nvp[similarities[sort_sim[1]]])
        except Exception:
            pass
    return result

In [ ]:
def most_similar_relation(relations, sentence, num):
    similarities = {}
    sen = concept_net(sentence)
    for i,rel in enumerate(relations):
        rel = concept_net(rel)
        sim = rel.similarity(sen)
        similarities[sim] = i
    sort_sim = sorted(similarities, reverse = True)
    # print(relations[similarities[sort_sim[0]]])
    result = []
    for i in range(num):
        result.append(relations[similarities[sort_sim[i]]])
    # result = [lambda x: relations[similarities[sort_sim[x]]] for x in range(num)]
    # print(result)
    return result

def word_bank(noun_phrases, sentence):
    relations = []
    for np in noun_phrases:
        words = np.split()
        for w in words:
            try:
                url = "http://api.conceptnet.io/c/en/@word?filter=/c/en"
                response = requests.get(url.replace("@word", w))
            
                obj = response.json()
                for edge in obj['edges']:
                    sen = None
                    if "start" in edge and "end" in edge and 'language' in edge['start'] and 'language' in edge['end']:
                        if edge['start']['language']=="en" and edge["end"]["language"]=="en":
                            sen = edge['surfaceText']
                    if sen is not None:
                        sen = ' '.join([word for word in sen.split() if word not in (stop_words_l)])
                        sen = sen.replace("[[", "")
                        sen = sen.replace("]]", "")
                        relations.append(sen)
            except Exception:
                pass

    return most_similar_relation(relations, sentence, 2)



In [ ]:
# def most_similar(doc_id,similarity_matrix,matrix,df):
#     if matrix=='Cosine Similarity':
#         similar_ix=np.argsort(similarity_matrix[doc_id])[::-1]
#     elif matrix=='Euclidean Distance':
#         similar_ix=np.argsort(similarity_matrix[doc_id])
#     return similar_ix[1:]
# def create_with_context(path, fname, num):
#   df = pd.read_csv(path)
#   content = []
#   for idx, row in df.iterrows():
#     ques = row['question']
#     passage = row['article']

#     ques = ques.replace('@placeholder', row["option_"+str(row["label"])])

#     docs = passage.split('. ')
#     docs.insert(0, ques)

#     temp_df = pd.DataFrame(docs,columns=['sentences'])
#     temp_df['cleaned']=temp_df.sentences.apply(lambda x: " ".join(re.sub(r'[^a-zA-Z]',' ',w).lower() for w in x.split() if re.sub(r'[^a-zA-Z]',' ',w).lower() not in stop_words_l))
    
#     document_embeddings = sbert_model.encode(temp_df['cleaned'])
#     pairwise_similarities=cosine_similarity(document_embeddings)

#     top_3 = most_similar(0,pairwise_similarities,'Cosine Similarity', temp_df)[:num]

#     len_ques = len(tokenizer.tokenize(row['question']))
#     sim_sent = [docs[x] for x in top_3[:2]]
#     len_pass = len(tokenizer.tokenize(' '.join(sim_sent)))

#     next_ind = 2
#     while next_ind<len(top_3) and len_pass+len_ques<=512:
#       sim_sent.append(docs[top_3[next_ind]])
#       len_pass = len(tokenizer.tokenize(' '.join(sim_sent)))
#       next_ind+=1

#     nv_phrases = extract_n_v_phrase(sim_sent)
#     relations = []
#     for p in nv_phrases:
#         relations.extend(word_bank(p, ' '.join(sim_sent)))

#     next_ind = 0
#     len_pass = len(tokenizer.tokenize(' '.join(sim_sent)))

#     while next_ind<len(relations) and len_pass+len_ques<=512:
#       sim_sent.append(relations[next_ind])
#       len_pass = len(tokenizer.tokenize(' '.join(sim_sent)))
#       next_ind+=1
#     sim_sent.pop()
#     ques = ' '.join(sim_sent) + row['question']
#     content.append(ques+'\n')

#   f = open(fname+'.txt', 'w')
#   f.writelines(content)
#   f.close()

In [ ]:
def most_similar(doc_id,similarity_matrix,matrix,df):
    if matrix=='Cosine Similarity':
        similar_ix=np.argsort(similarity_matrix[doc_id])[::-1]
    elif matrix=='Euclidean Distance':
        similar_ix=np.argsort(similarity_matrix[doc_id])
    return similar_ix[1:]
def create_with_context(path, fname, num):
  df = pd.read_csv(path)
  content = []
  for idx, row in df.iterrows():
    ques = row['question']
    passage = row['article']

    ques = ques.replace('@placeholder', row["option_"+str(row["label"])])

    docs = passage.split('. ')
    docs.insert(0, ques)

    temp_df = pd.DataFrame(docs,columns=['sentences'])
    temp_df['cleaned']=temp_df.sentences.apply(lambda x: " ".join(re.sub(r'[^a-zA-Z]',' ',w).lower() for w in x.split() if re.sub(r'[^a-zA-Z]',' ',w).lower() not in stop_words_l))
    
    document_embeddings = sbert_model.encode(temp_df['cleaned'])
    pairwise_similarities=cosine_similarity(document_embeddings)

    top_sen = most_similar(0,pairwise_similarities,'Cosine Similarity', temp_df)[:num]

    sim_sent = [docs[x] for x in top_sen]
    nv_phrases = extract_n_v_phrase(sim_sent)
    relations = []
    for p in nv_phrases:
        relations.extend(word_bank(p, ' '.join(sim_sent)))
    ques = ' '.join(sim_sent) + ' '.join(relations) + ques

    content.append(ques+'\n')
    
  f = open(fname+'.txt', 'w')
  f.writelines(content)
  f.close()

In [ ]:
sim_sent = ["Spain 's Marin beat Sindhu gold medal Olympic badminton singles final Rio de Janeiro  The Indian Twittersphere exploded Sindhu beat Marin 21 - 19 , 21 - 16 take India Open Super Series tournament Delhi night"]
noun_phrase = extract_n_v_phrase(sim_sent)
noun_phrase

In [ ]:
print(word_bank(noun_phrase, sim_sent[0]))

In [ ]:
path = "/content/train_task_1_new.csv"
create_with_context(path, "train_1", 2)

In [ ]:
path = "/content/dev_task_1.csv"
create_with_context(path, "dev_1", 2)

In [ ]:
!cp dev_1.txt /content/drive/My\ Drive/
# !cp train_1.txt /content/drive/My\ Drive/

In [ ]:
# !cp dev_1.txt /content/drive/My\ Drive/
!cp train_1.txt /content/drive/My\ Drive/

In [ ]:
!python run_mlm.py --model_name_or_path bert-base-cased --train_file '/content/train1.txt' --validation_file '/content/dev1.txt' --do_train --do_eval --line_by_line --output_dir /content/drive/MyDrive/task1_concept_net

In [ ]:
obj = requests.get('http://api.conceptnet.io/c/en/example').json()

In [ ]:
obj.keys()

In [ ]:
obj['edges'][2]

In [ ]:
sentence = "Ogwen Valley Mountain Rescue team , police RAF rescue helicopters took part search possible fallen walker Y Garn black white seen near Cwm Cywion gorge  Four members team climbed steep terrain rescue terrier"
doc = nlp(sentence) 
print(doc.ents)
# for token in doc: 
#   print(token, token.pos_) 
for token in doc: 
  print(token, token.pos_) 